In [5]:
import os
import re

def preprocess_markdown(text: str) -> str:
    """
    Функция для предобработки Markdown-текста.
    Выполняет следующие шаги:
      1. Заменяет блоки кода, заключённые в тройные обратные кавычки, на метку [CODE].
      2. Заменяет изображения на их alt-текст.
      3. Заменяет ссылки, оставляя только текст ссылки.
      4. Удаляет символы заголовков (#) в начале строки.
      5. Убирает маркеры маркированных (-, *) и нумерованных списков.
      6. Убирает разметку для выделения (жирный/курсив) – удаляет звездочки.
      7. Убирает обратные кавычки для встроенного кода.
      8. Убирает символы цитирования (>) в начале строки.
      9. Заменяет горизонтальные линии (например, '---') на разделитель '===='.
      10. Усечение ссылок: для каждого URL оставляет только часть до последнего символа "/".
      11. Удаляет раздел "Содержание" и всё, что к нему относится.
    """
    # 1. Заменяем блоки кода на метку [CODE]
    text = re.sub(r'```[\s\S]*?```', '[CODE]', text)
    # 2. Заменяем изображения на alt-текст
    text = re.sub(r'!\[([^\]]*)\]\([^)]*\)', r'\1', text)
    # 3. Заменяем ссылки, оставляя только текст ссылки
    text = re.sub(r'\[([^\]]+)\]\([^)]*\)', r'\1', text)
    # 4. Удаляем символы заголовков
    text = re.sub(r'^#+\s*', '', text, flags=re.MULTILINE)
    # 5. Обработка маркеров списков
    text = re.sub(r'^\s*[-*]\s+', '- ', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*\d+\.\s+', '', text, flags=re.MULTILINE)
    # 6. Убираем разметку жирного и курсива
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    # 7. Убираем обратные кавычки вокруг кода
    text = re.sub(r'`([^`]+)`', r'\1', text)
    # 8. Удаляем символы цитирования
    text = re.sub(r'^\s*>\s+', '', text, flags=re.MULTILINE)
    # 9. Заменяем горизонтальные линии на явный разделитель
    text = re.sub(r'^---\s*$', '====', text, flags=re.MULTILINE)
    # 10. Усечение ссылок: удаляем всё, что находится после последнего '/'
    text = trim_urls(text)
    # 11. Удаляем раздел "Содержание" и всё, что к нему относится
    text = re.sub(r'(^#+\s*Содержание.*?)(?=^#+\s*\S|$)', '', text, flags=re.DOTALL | re.MULTILINE | re.IGNORECASE)
    
    return text

def trim_urls(text: str) -> str:
    """
    Находит все URL в тексте (начинающиеся с http или https) и усечёт их,
    оставляя только часть до последнего символа "/". Например, из:
      https://sappo.ru/blog/.../21720/>  
    останется:
      https://sappo.ru/blog/.../21720/
    """
    # Шаблон, захватывающий URL до пробельного символа или знака конца строки
    url_pattern = re.compile(r'(https?://\S+)')
    
    def trim(match):
        url = match.group(0)
        last_slash = url.rfind('/')
        if last_slash != -1:
            return url[:last_slash+1]
        return url

    return url_pattern.sub(trim, text)

if __name__ == '__main__':
    input_dir = '/Users/sergey/Desktop/Voise_RAG/DATA/articles'
    
    # Обрабатываем все файлы с расширением .md в указанной директории
    for filename in os.listdir(input_dir):
        if filename.endswith('.md'):
            file_path = os.path.join(input_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                markdown_text = f.read()
            
            cleaned_text = preprocess_markdown(markdown_text)
            # Формируем имя выходного файла с суффиксом _cleaned, чтобы сохранить исходник нетронутым
            output_file = os.path.join(input_dir, filename.replace('.md', '_cleaned.md'))
            
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(cleaned_text)
            
            print(f'Обработан файл: {file_path}\nСохранён как: {output_file}\n')

Обработан файл: /Users/sergey/Desktop/Voise_RAG/DATA/articles/ukhod-za-kozhey.md
Сохранён как: /Users/sergey/Desktop/Voise_RAG/DATA/articles/ukhod-za-kozhey_cleaned.md

Обработан файл: /Users/sergey/Desktop/Voise_RAG/DATA/articles/khimchistka-salona-avtomobilya-svoimi-rukami.md
Сохранён как: /Users/sergey/Desktop/Voise_RAG/DATA/articles/khimchistka-salona-avtomobilya-svoimi-rukami_cleaned.md

Обработан файл: /Users/sergey/Desktop/Voise_RAG/DATA/articles/ukhod-za-plastikovymi-elementami-salona.md
Сохранён как: /Users/sergey/Desktop/Voise_RAG/DATA/articles/ukhod-za-plastikovymi-elementami-salona_cleaned.md

Обработан файл: /Users/sergey/Desktop/Voise_RAG/DATA/articles/kak-pravilno-otpolirovat-lobovoe-steklo.md
Сохранён как: /Users/sergey/Desktop/Voise_RAG/DATA/articles/kak-pravilno-otpolirovat-lobovoe-steklo_cleaned.md

Обработан файл: /Users/sergey/Desktop/Voise_RAG/DATA/articles/kak-izbavitsya-ot-zapaha-v-salone-avto.md
Сохранён как: /Users/sergey/Desktop/Voise_RAG/DATA/articles/kak-iz

In [6]:
import os
import shutil

# Директория с исходными файлами
input_dir = '/Users/sergey/Desktop/Voise_RAG/DATA/articles'
# Директория для сохранения файлов с окончанием "cleaned.md"
output_folder = os.path.join(input_dir, 'cleaned')
os.makedirs(output_folder, exist_ok=True)

# Перебор всех файлов в исходной директории
for filename in os.listdir(input_dir):
    if filename.endswith('cleaned.md'):
        src_file = os.path.join(input_dir, filename)
        dest_file = os.path.join(output_folder, filename)
        shutil.copy(src_file, dest_file)
        print(f'Файл {src_file} скопирован в {dest_file}')

Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/kak-ubrat-carapiny-s-plastika-salona-avtomobilya_cleaned.md скопирован в /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/kak-ubrat-carapiny-s-plastika-salona-avtomobilya_cleaned.md
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/vosk-dlya-avtomobilya-kak-ego-pravilno-nanosit-i-zachem-nuzhen_cleaned.md скопирован в /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/vosk-dlya-avtomobilya-kak-ego-pravilno-nanosit-i-zachem-nuzhen_cleaned.md
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/luchshie-aromatizatory-dlya-avtomobilya_cleaned.md скопирован в /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/luchshie-aromatizatory-dlya-avtomobilya_cleaned.md
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/mikrofibry-dlya-chego-nuzhny-kakie-byvayut-kak-pravilno-vybrat-i-kak-ukhazhivat_cleaned.md скопирован в /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/mikrofibry-dlya-chego-nuzhny-kakie-byvayut-kak-pravilno-vybrat-i-ka

In [7]:
def remove_duplicate_links(text: str) -> str:
    """
    Функция удаляет повторяющиеся URL в тексте, оставляя только первое вхождение каждого URL.
    Для поиска URL используется шаблон, который находит все подстроки, начинающиеся с http или https.
    """
    seen = set()

    def repl(match):
        url = match.group(0)
        if url in seen:
            # Если URL уже встречался, возвращаем пустую строку для удаления
            return ""
        seen.add(url)
        return url

    # Ищем все подстроки, соответствующие URL, и применяем замену через функцию repl
    return re.sub(r'(https?://\S+)', repl, text)

if __name__ == '__main__':
    # Директория с обработанными файлами, оканчивающимися на cleaned.md
    cleaned_dir = '/Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned'
    
    # Перебор всех файлов в указанной директории
    for filename in os.listdir(cleaned_dir):
        if filename.endswith('cleaned.md'):
            file_path = os.path.join(cleaned_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            new_content = remove_duplicate_links(content)
            
            # Перезаписываем файл новыми данными
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(new_content)
            
            print(f'Файл {file_path} обработан (удалены дубликаты ссылок).')

Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/kak-ubrat-carapiny-s-plastika-salona-avtomobilya_cleaned.md обработан (удалены дубликаты ссылок).
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/vosk-dlya-avtomobilya-kak-ego-pravilno-nanosit-i-zachem-nuzhen_cleaned.md обработан (удалены дубликаты ссылок).
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/luchshie-aromatizatory-dlya-avtomobilya_cleaned.md обработан (удалены дубликаты ссылок).
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/mikrofibry-dlya-chego-nuzhny-kakie-byvayut-kak-pravilno-vybrat-i-kak-ukhazhivat_cleaned.md обработан (удалены дубликаты ссылок).
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/ukhod-za-kozhey_cleaned.md обработан (удалены дубликаты ссылок).
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/samostoyatelnaya-obrabotka-lkp-voskom_cleaned.md обработан (удалены дубликаты ссылок).
Файл /Users/sergey/Desktop/Voise_RAG/DATA/articles/cleaned/abraziv